## Standard Problem

In [60]:
!pip install pulp
from pulp import *
from scipy.stats import norm

In [43]:
# model
m1 = LpProblem("Wyndor", LpMaximize)

In [44]:
# data
hours = {1: 4,
         2: 12,
         3: 18
        }

time = {'window': {1: 0, 2:2, 3:2},
        'door': {1: 1, 2:0, 3:3}
       }

profit = {'window': 5,
          'door': 3
         }

In [45]:
# variables 
x = LpVariable.dicts("production", profit, 0)

# objective function
m1 += lpSum([profit[item]*x[item] for item in profit])

# constraints
for plant in hours:
    m1 += lpSum([time[item][plant]*x[item] for item in profit]) <= hours[plant] ,"resource{}".format(plant)

In [46]:
# Optimize
m1.solve()
print("Status = %s" % LpStatus[m1.status])

# Solution
print("Objective = %f" % value(m1.objective))
for var in m1.variables():
    print("%s = %f" % (var,var.varValue))

Status = Optimal
Objective = 36.000000
production_door = 2.000000
production_window = 6.000000


## Robust Optimization

In [54]:
# model
m2 = LpProblem("WyndorRobust", LpMaximize)

In [55]:
# data
hours = {1: [3.6,4.4],
         2: [11,13],
         3: [16,20]
        }

time = {'window': {1: [0], 2:[1.8,2.2], 3:[1.5,2.5]},
        'door': {1: [0.8,1.2], 2:[0], 3:[2.5,3.5]}
       }

profit = {'window': [4.5,5.5],
          'door': [2.5,3.5]
         }

In [56]:
# variables 
x = LpVariable.dicts("production", profit, 0)

# objective function
m2 += lpSum([min(profit[item])*x[item] for item in profit])

# constraints
for plant in hours:
    m2 += lpSum([max(time[item][plant])*x[item] for item in profit]) <= min(hours[plant]) ,"resource{}".format(plant)

In [58]:
# Optimize
m2.solve()
print("Status = %s" % LpStatus[m2.status])

# Solution
print("Objective = %f" % value(m2.objective))
for var in m2.variables():
    print("%s = %f" % (var,var.varValue))

Status = Optimal
Objective = 25.000000
production_door = 1.000000
production_window = 5.000000


## Chance Constraints

In [80]:
# model
m3 = LpProblem("WyndorChance", LpMaximize)

In [81]:
# data
hours = {1: [4,0.2],
         2: [12,0.5],
         3: [18,1]
        }

time = {'window': {1: 0, 2:2, 3:2},
        'door': {1: 1, 2:0, 3:3}
       }

profit = {'window': 5,
          'door': 3
         }

In [83]:
# variables 
x = LpVariable.dicts("production", profit, 0)

# objective function
m3 += lpSum([profit[item]*x[item] for item in profit])

# constraints
for plant in hours:
    m3 += lpSum([time[item][plant]*x[item] for item in profit]) <= (hours[plant][0]-norm.ppf(0.95)*hours[plant][1]) ,"resource{}".format(plant)

In [85]:
# Optimize
m3.solve()
print("Status = %s" % LpStatus[m3.status])

# Solution
print("Objective = %f" % value(m3.objective))
for var in m3.variables():
    print("%s = %f" % (var,var.varValue))

Status = Optimal
Objective = 33.121506
production_door = 1.725858
production_window = 5.588787
